# Introduction

On s'intéresse ici à des clusters d'insectes pour identifier de potentielles régions endémiques. 
Suite à des erreurs mémoire dues au très grand nombre d'observations à traiter par DBSCAN, nous avons choisi de traiter des genres connus pour être de bons candidats à la recherche de régions endémiques : les genres Bombus et Apis de la famille des Apidae. 

# Chargement des données

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import folium
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("./spipoll.csv")

C:\Users\guilts0300\AppData\Local\Temp\ipykernel_19260\35506862.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./spipoll.csv")


## Préparation

In [3]:
# Séparation de la colonne 'coordonnées' en deux colonnes 'longitude' et 'latitude'
df[['longitude', 'latitude']] = df['coordonnees_GPS'].str.split(',', expand=True).astype(float)

# Conversion en datetime
df['collection_date'] = pd.to_datetime(df['collection_date'])

# Extraire l'année à partir de la colonne 'date_observation'
df['annee_collection'] = df['collection_date'].dt.year

In [4]:
unused_columns = ['collection_nom', 'collection_id', 'protocole_long', 'user_id',
       'photo_fleur', 'photo_plante', 'photo_feuille',               
       'plante_caractere', 'plante_inconnue',
       'photo_fleur', 'photo_plante', 'photo_feuille',
       'code_postal', 'fleur_ombre', 
       'insecte_abondance', 'insecte_commentaire', 'insecte_vu_sur_fleur',
        'grande_culture', 'habitat',
        'plante_famille', 'plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'collection_date',
       'collection_heure_debut', 'nebulosite', 'temperature', 'vent',   
        'coordonnees_GPS',
       'nb_validation', 'nb_suggestion']

In [5]:
df.drop(columns=unused_columns, inplace=True)

In [6]:
df.columns

Index(['insecte_ordre', 'insecte_super_famille', 'insecte_famille',
       'insecte_sous_famille', 'insecte_genre', 'insecte_espece', 'insecte_sc',
       'insecte_fr', 'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'longitude', 'latitude', 'annee_collection'],
      dtype='object')

## Filtrage du dataset

In [7]:
df['insecte_famille'].value_counts()

Apidae         85076
Syrphidae      80087
Halictidae     41163
Andrenidae     21857
Nymphalidae    21357
               ...  
Dascillidae       14
Acroceridae        5
Ptinidae           4
Alucitidae         4
Nolidae            1
Name: insecte_famille, Length: 120, dtype: int64

In [8]:
taxon='insecte_famille'

In [9]:
nom=['Apidae', 'Syrphidae']      

In [10]:
data=df.copy()
data = df.where(df[taxon].isin(nom)).dropna(subset=[taxon])
data.drop(columns=['insecte_ordre', 'insecte_super_famille', 'insecte_famille',
       'insecte_sous_famille', 'insecte_espece',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref'], inplace=True)

In [11]:
# Supprimer les lignes correspondant avec des années précédant 2011
data = data.drop(data[(data['annee_collection'] < 2011)].index)

# Réinitialiser les index après la suppression des lignes
data = data.reset_index(drop=True)

len(data)


160666

In [12]:
data['insecte_sc'].value_counts()

Bombus                            35864
Apis mellifera                    29328
Eristalis                         21649
Sphaerophoria                      9290
Episyrphus balteatus               7992
Meliscaeva et autres               5119
Syrphus, Dasysyrphus et autres     5037
Apidae et autres                   4936
Syritta                            3893
Eupeodes, Scaeva                   3875
Sphaerophoria scripta              3095
Myathropa florea                   2500
Nomada et autres                   2380
Helophilus, Parhelophilus          2130
Xylocopa                           1897
Anthophora, Eucera                 1576
Eristalinus                        1461
Xylocopa violacea                  1416
Merodon et autres                  1271
Chrysotoxum                        1052
Anthophora et autres               1041
Rhingia                             982
Ceratina cucurbitina                891
Amegilla et autres                  890
Eucera et autres                    790


# DBSCAN

In [13]:
# Créer un DataFrame avec uniquement les coordonnées GPS
coordonnees_gps = data[['latitude', 'longitude']]

# Instancier et entraîner le modèle DBSCAN
epsilon = 0.1  # Rayon maximal pour considérer les points voisins
min_samples = 400  # Nombre minimal de points dans un cluster
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
dbscan.fit(coordonnees_gps)

# Ajouter les labels de cluster au DataFrame d'origine
data['cluster_label'] = dbscan.labels_

# Filtrer les données pour exclure les points considérés comme du bruit (cluster_label = -1)
df_clusters = data[data['cluster_label'] != -1]

# Calculer la latitude et la longitude moyennes de chaque cluster
cluster_coords = df_clusters.groupby('cluster_label').agg({'latitude': 'mean', 'longitude': 'mean'})

# Calculer le nombre total d'insectes par cluster
total_insectes_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].count()

# Calculer la proportion de chaque type d'insecte par cluster
proportion_par_cluster = df_clusters.groupby(['cluster_label', 'insecte_fr']).size() / total_insectes_par_cluster
proportion_par_cluster = proportion_par_cluster.reset_index(name='proportion')

# Afficher les régions endémiques avec la proportion de chaque type d'insecte
regions = proportion_par_cluster.pivot(index='cluster_label', columns='insecte_fr', values='proportion').fillna(0)

# Ajouter les coordonnées de chaque cluster au DataFrame des régions endémiques
regions[['latitude', 'longitude']] = cluster_coords

# Ajouter le nombre d'insectes par clusters
regions['nb_insectes'] = total_insectes_par_cluster

print("Nombre de clusters : ", len(regions))


# Trouver l'insecte le plus abondant dans chaque cluster
insecte_majoritaire_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].agg(lambda x: x.value_counts().index[0])

# Calculer la proportion de l'insecte majoritaire dans chaque cluster
proportion_insecte_majoritaire_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].apply(lambda x: (x == x.mode()[0]).mean())


Nombre de clusters :  54


# Carte 

In [14]:
# Créer une fonction pour attribuer une couleur à chaque insecte
def assigner_couleur(insecte):
    if insecte == "L'Abeille mellifère":
        return 'darkred'
    elif insecte == 'Les Bourdons noirs à bande(s) jaune(s) et cul blanc':
        return 'lightred'
    elif insecte == 'Les Eristales (autres)':
        return 'green'
    #elif insecte == 'Les Bourdons noirs à bande(s) jaune(s) et cul rouge':
        #return 'lightgreen'
    else : 
        return 'blue'

In [15]:
# Coordonnées géographiques de Paris pour centrer la carte
paris_latitude = 48.8566
paris_longitude = 2.3522

# Création de la carte Folium centrée sur Paris
carte = folium.Map(location=[paris_latitude, paris_longitude], zoom_start=5)

# Ajouter des marqueurs pour chaque cluster
for index, row in cluster_coords.iterrows():
    insecte_majoritaire = insecte_majoritaire_par_cluster.loc[index]
    proportion_insecte_majoritaire = proportion_insecte_majoritaire_par_cluster.loc[index]
    nombre_d_insectes = total_insectes_par_cluster.loc[index]
    
    # Taille du marqueur proportionnelle au nombre d'insectes
    taille_marqueur = nombre_d_insectes / 10
    
    # Couleur du marqueur
    couleur_marqueur = assigner_couleur(insecte_majoritaire)
    
    # Créer le texte de la popup
    popup_text = f"Cluster {index} de {insecte_majoritaire}: Nombre d'insectes total = {nombre_d_insectes}, Proportion de {insecte_majoritaire} = {proportion_insecte_majoritaire:.2f}"
    
    # Ajouter le marqueur à la carte
    folium.Marker([row['longitude'], row['latitude']], popup=popup_text, icon=folium.Icon(color=couleur_marqueur), 
                  radius=taille_marqueur).add_to(carte)

# Afficher la carte
carte.save('carte_clusters_apidae_syrphidae_MS400.html')

In [17]:
carte

### Affinage de epsilon et min_sample

In [69]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Définir une plage de valeurs pour min_samples et epsilon
min_samples_range = [100, 500, 1000, 2000]
epsilon_range = np.linspace(0.01, 0.2, 20)

# Stocker le nombre de clusters pour chaque combinaison de valeurs
num_clusters = []

# Effectuer une analyse en grille pour chaque combinaison de min_samples et epsilon
for min_samples in min_samples_range:
    for epsilon in epsilon_range:
        dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
        dbscan.fit(data[['latitude', 'longitude']])
        num_clusters.append(len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0))  # Ignorer les points considérés comme du bruit

# Transformer la liste en une matrice 2D pour tracer le graphique
num_clusters = np.array(num_clusters).reshape(len(min_samples_range), len(epsilon_range))

# Tracer le graphique
plt.figure(figsize=(10, 6))
plt.imshow(num_clusters, extent=[min_samples_range[0], min_samples_range[-1], epsilon_range[0], epsilon_range[-1]], origin='lower', aspect='auto')
plt.colorbar(label='Nombre de clusters')
plt.xlabel('Min_samples')
plt.ylabel('Epsilon')
plt.title('Nombre de clusters en fonction de Min_samples et Epsilon')
plt.show()


KeyboardInterrupt: 

# Métriques d'évaluation

Mesurer la représentativité d'un cluster en tant que région endémique peut être un défi complexe, car cela nécessite de considérer plusieurs facteurs et caractéristiques à la fois spatiales et écologiques. 

1. **Densité de points** :
   - Utilisation de techniques de densité de noyaux (kernel density estimation) pour calculer la densité de points à l'intérieur de chaque cluster. Cela permettra d'obtenir une estimation de la densité de points par unité de surface dans chaque cluster.

2. **Variété d'espèces** :
   - Utilisation d'indices de diversité écologique tels que l'indice de Shannon-Wiener ou l'indice de Simpson pour évaluer la diversité des espèces présentes dans chaque cluster. Ces indices prennent en compte à la fois la richesse en espèces et l'équitabilité de leur distribution.

3. **Endémisme spécifique**:
   - Il pourrait être intéressant d'analyser les espèces endémiques présentes dans chaque cluster en comparant la liste des espèces dans le cluster avec une liste d'espèces connues pour être endémiques à la région d'intérêt. On pourrait alors calculer un indice d'endémisme spécifique pour chaque cluster en fonction du nombre et de la rareté des espèces endémiques qu'il contient.

4. **Stabilité temporelle** :
   - On aurait aimé utilisé des méthodes d'analyse de série chronologique pour évaluer la persistance des clusters dans différentes périodes de temps, notamment à l'aide de ST-DBSCAN, que nous n'avons pas réussi à installer. On pourrait également comparer la taille et la forme des clusters dans des périodes temporelles différentes pour identifier les clusters qui persistent sur le long terme.

5. **Isolation géographique** :
   - Les clusters situés dans des zones géographiquement isolées peuvent être considérés comme plus importants pour la conservation. On peut calculer la distince au cluster le plus proche ou calculer des mesures d'isolation géographique pour chaque cluster, telles que la distance aux clusters voisins ou l'indice de Moran's I pour évaluer la spatialité des clusters.


Ultimement, on pourrait pondérer et agréger ces mesures pour combiner ces mesures en une seule métrique de représentativité. 

## Densité des clusters 

In [70]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

data2=data.copy()
data2.drop(columns='annee_collection', inplace=True)
# Créer des géométries ponctuelles à partir des colonnes de latitude et de longitude
geometry = [Point(xy) for xy in zip(data2.longitude, data2.latitude)]

# Créer un GeoDataFrame à partir des données et des géométries
gdf = gpd.GeoDataFrame(data2, geometry=geometry)

# Définir le système de coordonnées de référence (CRS) si nécessaire
gdf.crs = "EPSG:4326"  # Par exemple, WGS 84

# Enregistrer le GeoDataFrame dans un fichier
gdf.to_file("clusters_gdf.shp")  # Enregistrez les données au format Shapefile


C:\Users\guilts0300\AppData\Local\Temp\ipykernel_21092\3732293045.py:17: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("clusters_gdf.shp")  # Enregistrez les données au format Shapefile


In [71]:
from sklearn.neighbors import KernelDensity

# Calculer les centres des clusters en utilisant la moyenne des coordonnées
cluster_centers = df_clusters.groupby('cluster_label').agg({'latitude': 'mean', 'longitude': 'mean'})

# Créer un tableau Numpy des coordonnées des centres des clusters
cluster_coordinates = cluster_centers.values

# Créer un modèle de densité de noyaux avec Scikit-learn
bandwidth = 0.1  # A régler selon les besoins
kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')

# Adapter le modèle aux coordonnées des centres des clusters
kde.fit(cluster_coordinates)

# Évaluer la densité de points pour chaque centre de cluster
densities = np.exp(kde.score_samples(cluster_coordinates))

# Ajouter les densités calculées au DataFrame des clusters
cluster_centers['densite_cluster'] = densities

regions['densite_cluster']=densities
regions

insecte_fr,L'Abeille Ceratina noire,L'Abeille coucou Epeloides (femelle),L'Abeille mellifère,L'Eristale des fleurs,L'Eristale taeniops,La Baccha allongée,La Ferdinandea cuprea,La Milésie bigarée,La Milésie bourdon,La Milésie frelon,...,Les Syrphes aux fémurs enflés,Les Syrphes difficiles à déterminer,Les Syrphes à abdomen fin,Les Syrphes à l'aspect de bourdon,Les Syrphes à taches en virgules,Les Xylocopes,latitude,longitude,nb_insectes,densite_cluster
cluster_label,,,,,,,,,,,,,,,,,,,,,
0,0.002981,0.000221,0.187645,0.020702,0.000055,0.000939,0.000331,0.000000,0.000000,0.000000,...,0.004140,0.038258,0.030087,0.008336,0.034835,0.005245,2.180222,48.895483,18114,0.353678
1,0.012114,0.000000,0.292398,0.017962,0.019215,0.000000,0.000418,0.000000,0.000000,0.000835,...,0.001671,0.017962,0.028404,0.004177,0.025480,0.024227,5.911147,43.208053,2394,0.353678
2,0.001076,0.000098,0.145660,0.015021,0.000000,0.000049,0.000342,0.000000,0.000000,0.000000,...,0.008514,0.043106,0.035424,0.008171,0.015217,0.003034,-1.669293,48.201629,20438,0.353678
3,0.015723,0.000000,0.222322,0.012872,0.000326,0.000081,0.003747,0.000000,0.000000,0.001548,...,0.002525,0.050265,0.031202,0.004318,0.023625,0.025173,4.589101,44.701850,12275,0.354169
4,0.003677,0.000000,0.212552,0.021574,0.004413,0.000000,0.000245,0.000000,0.000000,0.001226,...,0.009561,0.028684,0.027703,0.007355,0.013974,0.045354,4.399900,43.974169,4079,0.353678
5,0.010024,0.000000,0.155012,0.008802,0.000000,0.000000,0.001467,0.000000,0.000000,0.006357,...,0.004401,0.035208,0.029829,0.002445,0.015403,0.012714,1.020606,44.993360,4090,0.353678
6,0.000000,0.000000,0.098039,0.019608,0.007541,0.000000,0.000000,0.000000,0.000000,0.001508,...,0.007541,0.054299,0.033183,0.001508,0.039216,0.006033,-1.738195,43.373781,663,0.353678
7,0.001426,0.000000,0.114754,0.032787,0.005702,0.000000,0.000713,0.000000,0.000713,0.007128,...,0.000713,0.057733,0.045617,0.013542,0.006415,0.009979,-0.264680,44.808205,1403,0.355715
8,0.000498,0.000000,0.170404,0.016941,0.000000,0.000498,0.001495,0.000000,0.000000,0.000997,...,0.003488,0.026408,0.027404,0.008969,0.013453,0.022422,2.003263,44.603844,2007,0.367070


## Variétés d'espèces

In [79]:
regions.iloc[:, :54]

insecte_fr,L'Abeille Ceratina noire,L'Abeille coucou Epeloides (femelle),L'Abeille mellifère,L'Eristale des fleurs,L'Eristale taeniops,La Baccha allongée,La Ferdinandea cuprea,La Milésie bigarée,La Milésie bourdon,La Milésie frelon,...,Les Syrphes Sphaerophoria (femelle),Les Syrphes Sphaerophoria (mâle),Les Syrphes Syrphus et autres,Les Syrphes aux antennes à bout blanc,Les Syrphes aux fémurs enflés,Les Syrphes difficiles à déterminer,Les Syrphes à abdomen fin,Les Syrphes à l'aspect de bourdon,Les Syrphes à taches en virgules,Les Xylocopes
cluster_label,,,,,,,,,,,,,,,,,,,,,
0,0.002981,0.000221,0.187645,0.020702,0.000055,0.000939,0.000331,0.000000,0.000000,0.000000,...,0.048526,0.020261,0.033786,0.000276,0.004140,0.038258,0.030087,0.008336,0.034835,0.005245
1,0.012114,0.000000,0.292398,0.017962,0.019215,0.000000,0.000418,0.000000,0.000000,0.000835,...,0.032581,0.008354,0.011696,0.000000,0.001671,0.017962,0.028404,0.004177,0.025480,0.024227
2,0.001076,0.000098,0.145660,0.015021,0.000000,0.000049,0.000342,0.000000,0.000000,0.000000,...,0.025247,0.012721,0.030140,0.000000,0.008514,0.043106,0.035424,0.008171,0.015217,0.003034
3,0.015723,0.000000,0.222322,0.012872,0.000326,0.000081,0.003747,0.000000,0.000000,0.001548,...,0.047902,0.021426,0.023788,0.000733,0.002525,0.050265,0.031202,0.004318,0.023625,0.025173
4,0.003677,0.000000,0.212552,0.021574,0.004413,0.000000,0.000245,0.000000,0.000000,0.001226,...,0.034322,0.012258,0.023535,0.000245,0.009561,0.028684,0.027703,0.007355,0.013974,0.045354
5,0.010024,0.000000,0.155012,0.008802,0.000000,0.000000,0.001467,0.000000,0.000000,0.006357,...,0.054768,0.023961,0.024450,0.000489,0.004401,0.035208,0.029829,0.002445,0.015403,0.012714
6,0.000000,0.000000,0.098039,0.019608,0.007541,0.000000,0.000000,0.000000,0.000000,0.001508,...,0.095023,0.034691,0.024133,0.000000,0.007541,0.054299,0.033183,0.001508,0.039216,0.006033
7,0.001426,0.000000,0.114754,0.032787,0.005702,0.000000,0.000713,0.000000,0.000713,0.007128,...,0.063435,0.005702,0.037776,0.002138,0.000713,0.057733,0.045617,0.013542,0.006415,0.009979
8,0.000498,0.000000,0.170404,0.016941,0.000000,0.000498,0.001495,0.000000,0.000000,0.000997,...,0.021923,0.002491,0.029895,0.002491,0.003488,0.026408,0.027404,0.008969,0.013453,0.022422


In [72]:
# Les 54 premières colonnes du dataset regions correspondent aux proportions d'insectes
colonnes_proportions=54

In [73]:
# Calculer l'indice de Shannon-Wiener pour chaque cluster
def shannon_index(proportions):
    return -np.sum(proportions * np.log(proportions + 1e-10))  # Ajout de 1e-10 pour éviter les erreurs de division par zéro

# Calculer l'indice de Simpson pour chaque cluster
def simpson_index(proportions):
    return 1 - np.sum(proportions ** 2)

# Calculer l'indice de Shannon-Wiener pour chaque cluster
regions['shannon_index'] = regions.iloc[:, :colonnes_proportions].apply(shannon_index, axis=1)  # Supposant que les colonnes 0 à 7 contiennent les proportions des insectes

# Calculer l'indice de Simpson pour chaque cluster
regions['simpson_index'] = regions.iloc[:, :colonnes_proportions].apply(simpson_index, axis=1)  # Supposant que les colonnes 0 à 7 contiennent les proportions des insectes

# Afficher les indices de Shannon-Wiener et de Simpson pour chaque cluster
regions[['shannon_index', 'simpson_index']]


insecte_fr,shannon_index,simpson_index
cluster_label,,
0,2.979063,0.924372
1,2.742507,0.877743
2,2.869579,0.910696
3,3.003263,0.915035
4,3.013930,0.920111
5,3.053615,0.929144
6,2.925288,0.928064
7,3.044721,0.928414
8,2.963366,0.915404


Interprétations : 

- Indice de Shannon-Wiener :

Plus l'indice de Shannon-Wiener est élevé, plus la diversité des espèces dans le cluster est grande.
Cela signifie qu'il y a une grande variété d'espèces présentes dans le cluster, et que leur abondance est relativement uniforme.
Un indice de Shannon-Wiener proche de zéro indique une faible diversité des espèces, ce qui peut être dû à la dominance d'une ou de quelques espèces dans le cluster.

- Indice de Simpson : L'indice de Simpson est un indice permettant de mesurer la diversité d'un milieu, créé par Edward Simpson en 1949, en calculant la probabilité que deux individus sélectionnés au hasard appartiennent à la même espèce. 

Un indice de Simpson proche de 1 indique une faible diversité des espèces, avec une forte dominance d'une ou de quelques espèces dans le cluster.
Plus l'indice de Simpson est proche de zéro, plus la diversité des espèces est élevée, car les espèces sont plus équitablement réparties dans le cluster.
Ainsi, un indice de Simpson faible indique une grande diversité des espèces dans le cluster, avec une répartition plus uniforme des abondances des espèces.
En résumé, ces indices permettent d'évaluer la diversité des espèces dans un cluster en prenant en compte à la fois le nombre d'espèces présentes et la répartition relative de leur abondance. Ils fournissent des informations précieuses sur la structure et la composition de la communauté d'espèces dans un cluster donné. Une interprétation approfondie nécessiterait de comparer ces indices entre différents clusters ou à des échantillons de référence pour évaluer la santé et la biodiversité de l'écosystème étudié.

## Isolation géographique

In [74]:
# Calcul de la distance en km séparant 2 clusters les plus proches

from math import radians, sin, cos, sqrt, atan2

# Fonction pour calculer la distance en kilomètres entre deux points en utilisant la formule de Haversine
def distance_haversine(lat1, lon1, lat2, lon2):
    # Convertir les coordonnées de degrés en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Rayon moyen de la Terre en kilomètres
    R = 6371.0

    # Calcul des différences de latitude et de longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calcul de la distance géodésique en utilisant la formule de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance


# Créer une nouvelle colonne pour stocker les distances vers le cluster le plus proche
regions['km_du_cluster_le_plus_proche'] = 0.0

# Pour chaque cluster
for i, row in regions.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    min_distance = float('inf')  # Initialiser avec une valeur infinie pour trouver la distance minimale

    # Calculer la distance vers chaque autre cluster
    for j, other_row in regions.iterrows():
        if i != j:  # Pour éviter de calculer la distance à lui-même
            lat2, lon2 = other_row['latitude'], other_row['longitude']
            distance = distance_haversine(lat1, lon1, lat2, lon2)
            min_distance = min(min_distance, distance)  # Mettre à jour la distance minimale
    
    # Assigner la distance minimale au cluster le plus proche
    regions.at[i, 'km_du_cluster_le_plus_proche'] = min_distance

# Afficher le DataFrame mis à jour
regions

insecte_fr,L'Abeille Ceratina noire,L'Abeille coucou Epeloides (femelle),L'Abeille mellifère,L'Eristale des fleurs,L'Eristale taeniops,La Baccha allongée,La Ferdinandea cuprea,La Milésie bigarée,La Milésie bourdon,La Milésie frelon,...,Les Syrphes à l'aspect de bourdon,Les Syrphes à taches en virgules,Les Xylocopes,latitude,longitude,nb_insectes,densite_cluster,shannon_index,simpson_index,km_du_cluster_le_plus_proche
cluster_label,,,,,,,,,,,,,,,,,,,,,
0,0.002981,0.000221,0.187645,0.020702,0.000055,0.000939,0.000331,0.000000,0.000000,0.000000,...,0.008336,0.034835,0.005245,2.180222,48.895483,18114,0.353678,2.979063,0.924372,75.075260
1,0.012114,0.000000,0.292398,0.017962,0.019215,0.000000,0.000418,0.000000,0.000000,0.000835,...,0.004177,0.025480,0.024227,5.911147,43.208053,2394,0.353678,2.742507,0.877743,62.588729
2,0.001076,0.000098,0.145660,0.015021,0.000000,0.000049,0.000342,0.000000,0.000000,0.000000,...,0.008171,0.015217,0.003034,-1.669293,48.201629,20438,0.353678,2.869579,0.910696,77.723602
3,0.015723,0.000000,0.222322,0.012872,0.000326,0.000081,0.003747,0.000000,0.000000,0.001548,...,0.004318,0.023625,0.025173,4.589101,44.701850,12275,0.354169,3.003263,0.915035,40.216417
4,0.003677,0.000000,0.212552,0.021574,0.004413,0.000000,0.000245,0.000000,0.000000,0.001226,...,0.007355,0.013974,0.045354,4.399900,43.974169,4079,0.353678,3.013930,0.920111,58.122741
5,0.010024,0.000000,0.155012,0.008802,0.000000,0.000000,0.001467,0.000000,0.000000,0.006357,...,0.002445,0.015403,0.012714,1.020606,44.993360,4090,0.353678,3.053615,0.929144,117.531922
6,0.000000,0.000000,0.098039,0.019608,0.007541,0.000000,0.000000,0.000000,0.000000,0.001508,...,0.001508,0.039216,0.006033,-1.738195,43.373781,663,0.353678,2.925288,0.928064,144.518182
7,0.001426,0.000000,0.114754,0.032787,0.005702,0.000000,0.000713,0.000000,0.000713,0.007128,...,0.013542,0.006415,0.009979,-0.264680,44.808205,1403,0.355715,3.044721,0.928414,35.709640
8,0.000498,0.000000,0.170404,0.016941,0.000000,0.000498,0.001495,0.000000,0.000000,0.000997,...,0.008969,0.013453,0.022422,2.003263,44.603844,2007,0.367070,2.963366,0.915404,28.435150


### Indice de Moran's I 

In [75]:
#!pip install libpysal
#!pip install pysal

In [46]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

data2=data.copy()
data2.drop(columns='annee_collection', inplace=True)
# Créer des géométries ponctuelles à partir des colonnes de latitude et de longitude
geometry = [Point(xy) for xy in zip(data2.longitude, data2.latitude)]

# Créer un GeoDataFrame à partir des données et des géométries
gdf = gpd.GeoDataFrame(data2, geometry=geometry)

# Définir le système de coordonnées de référence (CRS) si nécessaire
gdf.crs = "EPSG:4326"  # Par exemple, WGS 84

# Enregistrer le GeoDataFrame dans un fichier
gdf.to_file("data_MoransI.shp")  # Enregistrez les données au format Shapefile


C:\Users\guilts0300\AppData\Local\Temp\ipykernel_17992\3505635977.py:17: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("data_MoransI.shp")  # Enregistrez les données au format Shapefile


In [47]:
import geopandas as gpd
from libpysal.weights import Queen
from esda.moran import Moran

# Charger les données géographiques des clusters
clusters_gdf = gpd.read_file("data_MoransI.shp")  # Remplacez "clusters.shp" par le chemin vers votre fichier de clusters

# Calculer la matrice de pondération spatiale (matrice des voisins)
w = Queen.from_dataframe(clusters_gdf)

# Calculer l'indice de Moran's I pour les valeurs de chaque cluster (par exemple, la densité de points)
moran = Moran(clusters_gdf['densite_points'], w)

# Afficher le résultat
print("Moran's I:", moran.I)
print("P-value:", moran.p_sim)


C:\Users\guilts0300\AppData\Local\Temp\ipykernel_17992\3010085530.py:9: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(clusters_gdf)


KeyError: 17381

## Stabilité temporelle

On aurait aimé utiliser ST-DBSCAN mais problème d'installation de sk mobility. On aurait également souahité utiliser l'API de arcgis Pro qui permet d'évaluer les clusters dans le temps.

In [76]:
# Convertir la colonne de date en format datetime
#data['date_observation'] = pd.to_datetime(data['date_observation'])

# Calculer la stabilité temporelle pour chaque cluster
cluster_stability = df_clusters.groupby('cluster_label').apply(lambda x: x['annee_collection'].max() - x['annee_collection'].min())

# Calculer la dernière année observée pour chaque cluster
last_obs_year = df_clusters.groupby('cluster_label')['annee_collection'].max()

# Afficher les résultats
regions['cluster_stability'] = cluster_stability
regions['last_obs_year'] = last_obs_year
regions

insecte_fr,L'Abeille Ceratina noire,L'Abeille coucou Epeloides (femelle),L'Abeille mellifère,L'Eristale des fleurs,L'Eristale taeniops,La Baccha allongée,La Ferdinandea cuprea,La Milésie bigarée,La Milésie bourdon,La Milésie frelon,...,Les Xylocopes,latitude,longitude,nb_insectes,densite_cluster,shannon_index,simpson_index,km_du_cluster_le_plus_proche,cluster_stability,last_obs_year
cluster_label,,,,,,,,,,,,,,,,,,,,,
0,0.002981,0.000221,0.187645,0.020702,0.000055,0.000939,0.000331,0.000000,0.000000,0.000000,...,0.005245,2.180222,48.895483,18114,0.353678,2.979063,0.924372,75.075260,12.0,2023.0
1,0.012114,0.000000,0.292398,0.017962,0.019215,0.000000,0.000418,0.000000,0.000000,0.000835,...,0.024227,5.911147,43.208053,2394,0.353678,2.742507,0.877743,62.588729,12.0,2023.0
2,0.001076,0.000098,0.145660,0.015021,0.000000,0.000049,0.000342,0.000000,0.000000,0.000000,...,0.003034,-1.669293,48.201629,20438,0.353678,2.869579,0.910696,77.723602,12.0,2023.0
3,0.015723,0.000000,0.222322,0.012872,0.000326,0.000081,0.003747,0.000000,0.000000,0.001548,...,0.025173,4.589101,44.701850,12275,0.354169,3.003263,0.915035,40.216417,12.0,2023.0
4,0.003677,0.000000,0.212552,0.021574,0.004413,0.000000,0.000245,0.000000,0.000000,0.001226,...,0.045354,4.399900,43.974169,4079,0.353678,3.013930,0.920111,58.122741,12.0,2023.0
5,0.010024,0.000000,0.155012,0.008802,0.000000,0.000000,0.001467,0.000000,0.000000,0.006357,...,0.012714,1.020606,44.993360,4090,0.353678,3.053615,0.929144,117.531922,12.0,2023.0
6,0.000000,0.000000,0.098039,0.019608,0.007541,0.000000,0.000000,0.000000,0.000000,0.001508,...,0.006033,-1.738195,43.373781,663,0.353678,2.925288,0.928064,144.518182,12.0,2023.0
7,0.001426,0.000000,0.114754,0.032787,0.005702,0.000000,0.000713,0.000000,0.000713,0.007128,...,0.009979,-0.264680,44.808205,1403,0.355715,3.044721,0.928414,35.709640,12.0,2023.0
8,0.000498,0.000000,0.170404,0.016941,0.000000,0.000498,0.001495,0.000000,0.000000,0.000997,...,0.022422,2.003263,44.603844,2007,0.367070,2.963366,0.915404,28.435150,10.0,2022.0


#### Génération cartes annuelles

##### Préparation

In [80]:
data.drop(columns=['cluster_label'], inplace=True)

In [81]:
data

,insecte_genre,insecte_sc,insecte_fr,longitude,latitude,annee_collection
0,Apis,Apis mellifera,L'Abeille mellifère,45.363808,6.514947,2019.0
1,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,45.363808,6.514947,2019.0
2,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,48.844975,2.358313,2019.0
3,Apis,Apis mellifera,L'Abeille mellifère,48.844975,2.358313,2019.0
4,Apis,Apis mellifera,L'Abeille mellifère,47.679478,-3.183325,2019.0
...,...,...,...,...,...,...
160661,Eristalis,Eristalis,Les Eristales (autres),44.712529,4.568813,2023.0
160662,Sphaerophoria,Sphaerophoria,Les Syrphes Sphaerophoria (femelle),44.712529,4.568813,2023.0
160663,Eristalis,Eristalis,Les Eristales (autres),44.712618,4.568999,2023.0
160664,Bombus,Bombus,Les Bourdons à pilosité fauve à grise,44.712618,4.568999,2023.0


In [82]:
# Compter le nombre d'observations par année
observations_par_annee = data['annee_collection'].value_counts()

observations_par_annee

2021.0    18812
2020.0    17604
2019.0    13582
2022.0    13402
2023.0    12826
2016.0    12669
2013.0    11989
2018.0    11618
2015.0    11356
2017.0    10948
2014.0     9367
2011.0     8288
2012.0     8205
Name: annee_collection, dtype: int64

In [83]:
data.to_csv('data_clustering_et_metrique.csv', index=False) 

##### Relance analyse

In [84]:
# Créer une fonction pour attribuer une couleur à chaque insecte
def assigner_couleur(insecte):
    if insecte == "L'Abeille mellifère":
        return 'darkred'
    elif insecte == 'Les Bourdons noirs à bande(s) jaune(s) et cul blanc':
        return 'lightred'
    elif insecte == 'Les Eristales (autres)':
        return 'green'
    #elif insecte == 'Les Bourdons noirs à bande(s) jaune(s) et cul rouge':
        #return 'lightgreen'
    else : 
        return 'blue'

In [88]:
from sklearn.cluster import DBSCAN
import pandas as pd
import folium

data = pd.read_csv("./data_clustering_et_metrique.csv")  

# Pour chaque année unique dans la colonne 'annee_collection'
for annee in data['annee_collection'].unique():
    # Filtrer les données pour l'année spécifique
    data_annee = data[data['annee_collection'] == annee]
    
    # Créer un DataFrame avec uniquement les coordonnées GPS
    coordonnees_gps = data_annee[['latitude', 'longitude']]

    # Instancier et entraîner le modèle DBSCAN
    epsilon = 0.1  # Rayon maximal pour considérer les points voisins
    min_samples = 500  # Nombre minimal de points dans un cluster
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    dbscan.fit(coordonnees_gps)

    # Ajouter les labels de cluster au DataFrame d'origine
    data_annee['cluster_label'] = dbscan.labels_

    # Filtrer les données pour exclure les points considérés comme du bruit (cluster_label = -1)
    df_clusters = data_annee[data_annee['cluster_label'] != -1]

    # Calculer la latitude et la longitude moyennes de chaque cluster
    cluster_coords = df_clusters.groupby('cluster_label').agg({'latitude': 'mean', 'longitude': 'mean'})

    # Calculer le nombre total d'insectes par cluster
    total_insectes_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].count()

    # Calculer la proportion de chaque type d'insecte par cluster
    proportion_par_cluster = df_clusters.groupby(['cluster_label', 'insecte_fr']).size() / total_insectes_par_cluster
    proportion_par_cluster = proportion_par_cluster.reset_index(name='proportion')

    # Afficher les régions endémiques avec la proportion de chaque type d'insecte
    regions = proportion_par_cluster.pivot(index='cluster_label', columns='insecte_fr', values='proportion').fillna(0)

    # Ajouter les coordonnées de chaque cluster au DataFrame des régions endémiques
    regions[['latitude', 'longitude']] = cluster_coords

    # Ajouter le nombre d'insectes par clusters
    regions['nb_insectes'] = total_insectes_par_cluster

    # Trouver l'insecte le plus abondant dans chaque cluster
    insecte_majoritaire_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].agg(lambda x: x.value_counts().index[0])

    # Calculer la proportion de l'insecte majoritaire dans chaque cluster
    proportion_insecte_majoritaire_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].apply(lambda x: (x == x.mode()[0]).mean())

    # Coordonnées géographiques de Paris pour centrer la carte
    paris_latitude = 48.8566
    paris_longitude = 2.3522

    # Création de la carte Folium centrée sur Paris
    carte = folium.Map(location=[paris_latitude, paris_longitude], zoom_start=5)

    # Ajouter des marqueurs pour chaque cluster
    for index, row in cluster_coords.iterrows():
        insecte_majoritaire = insecte_majoritaire_par_cluster.loc[index]
        proportion_insecte_majoritaire = proportion_insecte_majoritaire_par_cluster.loc[index]
        nombre_d_insectes = total_insectes_par_cluster.loc[index]

        # Taille du marqueur proportionnelle au nombre d'insectes
        taille_marqueur = nombre_d_insectes / 10

        # Couleur du marqueur
        couleur_marqueur = assigner_couleur(insecte_majoritaire)

        # Créer le texte de la popup
        popup_text = f"Cluster {index} de {insecte_majoritaire}: Nombre d'insectes total = {nombre_d_insectes}, Proportion de {insecte_majoritaire} = {proportion_insecte_majoritaire:.2f}"

        # Ajouter le marqueur à la carte
        folium.Marker([row['longitude'], row['latitude']], popup=popup_text, icon=folium.Icon(color=couleur_marqueur), 
                      radius=taille_marqueur).add_to(carte)

    # Sauvegarde de la carte pour cette année dans le répertoire spécifique
    carte.save(f'./cartes_clustering/carte_clusters_apidae_syrphidae{annee}.html')

C:\Users\guilts0300\AppData\Local\Temp\ipykernel_21092\1014401864.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_annee['cluster_label'] = dbscan.labels_
C:\Users\guilts0300\AppData\Local\Temp\ipykernel_21092\1014401864.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_annee['cluster_label'] = dbscan.labels_
C:\Users\guilts0300\AppData\Local\Temp\ipykernel_21092\1014401864.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [92]:
import os
import imageio
from selenium import webdriver
import os

# Répertoire contenant les cartes Folium
repertoire_cartes = './cartes_clustering'

# Liste pour stocker les chemins des fichiers des cartes Folium
chemins_cartes = []

# Récupérer les chemins des fichiers des cartes Folium
for fichier in os.listdir(repertoire_cartes):
    if fichier.endswith('.html'):
        chemin_fichier = os.path.join(repertoire_cartes, fichier)
        chemins_cartes.append(chemin_fichier)

# Triez les chemins des fichiers par ordre alphabétique pour garantir l'ordre chronologique
chemins_cartes.sort()

# Liste pour stocker les images des cartes
images = []

# Conversion des cartes en images
for chemin_carte in chemins_cartes:
    # Ouvrir le fichier HTML et le lire en tant que texte brut
    with open(chemin_carte, 'r') as f:
        html_content = f.read()

        # Créer un répertoire pour sauvegarder les images temporaires
        os.makedirs('images_temp', exist_ok=True)

        # Configuration du navigateur
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Pour exécuter Chrome en mode headless (sans interface graphique)
        driver = webdriver.Chrome(options=options)
        
        # Parcourir les fichiers HTML des cartes Folium et prendre des captures d'écran
        for chemin_carte in chemins_cartes:
            # Charger le fichier HTML dans le navigateur
            driver.get('file:///' + os.path.abspath(chemin_carte))

            # Prendre une capture d'écran et sauvegarder l'image
            image_name = os.path.basename(chemin_carte)[:-5] + '.png'  # Nom de l'image basé sur le nom du fichier HTML
            image_path = os.path.join('cartes_clustering', image_name)  # Chemin complet pour enregistrer l'image
            driver.save_screenshot(image_path)

            # Ajouter le chemin de l'image à la liste des images
            images.append(image_path)

# Fermer le navigateur
driver.quit()

## Analyse à partir des métriques

Génération des cartes clusterisées années par années : 
- on voit 

In [95]:
# Grouper le DataFrame par année de collection et compter le nombre d'observations par année
grouped_data = data.groupby('annee_collection').size().reset_index(name='nombre_observations')

# Afficher le DataFrame groupé
grouped_data


,annee_collection,nombre_observations
0,2011.0,8288
1,2012.0,8205
2,2013.0,11989
3,2014.0,9367
4,2015.0,11356
5,2016.0,12669
6,2017.0,10948
7,2018.0,11618
8,2019.0,13582
9,2020.0,17604


In [22]:
# Filtrer des points considérés comme du bruit (cluster_label = -1)
df_bruit = data[data['cluster_label'] == -1]

In [23]:
df_bruit

,insecte_genre,insecte_sc,insecte_fr,longitude,latitude,annee_collection,cluster_label
0,Apis,Apis mellifera,L'Abeille mellifère,45.363808,6.514947,2019.0,-1
1,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,45.363808,6.514947,2019.0,-1
8,Apis,Apis mellifera,L'Abeille mellifère,47.679478,-3.183325,2019.0,-1
109,Apis,Apis mellifera,L'Abeille mellifère,43.086203,6.021628,2019.0,-1
110,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,43.086203,6.021628,2019.0,-1
...,...,...,...,...,...,...,...
670310,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,46.096694,2.061500,2023.0,-1
670629,Bombus,Bombus,Les Bourdons noirs à cul rouge,49.337282,4.185961,2023.0,-1
670639,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,49.337134,4.187314,2023.0,-1
670640,Bombus,Bombus,Les Bourdons noirs à cul rouge,49.337134,4.187314,2023.0,-1


In [ ]:
# Calculer la latitude et la longitude moyennes de chaque cluster
cluster_coords = df_bruit.groupby('cluster_label').agg({'latitude': 'mean', 'longitude': 'mean'})

# Calculer le nombre total d'insectes par cluster
total_insectes_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].count()

# Calculer la proportion de chaque type d'insecte par cluster
proportion_par_cluster = df_clusters.groupby(['cluster_label', 'insecte_fr']).size() / total_insectes_par_cluster
proportion_par_cluster = proportion_par_cluster.reset_index(name='proportion')

# Afficher les régions endémiques avec la proportion de chaque type d'insecte
regions = proportion_par_cluster.pivot(index='cluster_label', columns='insecte_fr', values='proportion').fillna(0)

# Ajouter les coordonnées de chaque cluster au DataFrame des régions endémiques
regions[['latitude', 'longitude']] = cluster_coords

# Ajouter le nombre d'insectes par clusters
regions['nb_insectes'] = total_insectes_par_cluster

print("Nombre de clusters : ", len(regions))


# Trouver l'insecte le plus abondant dans chaque cluster
insecte_majoritaire_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].agg(lambda x: x.value_counts().index[0])

# Calculer la proportion de l'insecte majoritaire dans chaque cluster
proportion_insecte_majoritaire_par_cluster = df_clusters.groupby('cluster_label')['insecte_fr'].apply(lambda x: (x == x.mode()[0]).mean())

In [21]:
data

,insecte_genre,insecte_sc,insecte_fr,longitude,latitude,annee_collection,cluster_label
0,Apis,Apis mellifera,L'Abeille mellifère,45.363808,6.514947,2019.0,-1
1,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,45.363808,6.514947,2019.0,-1
2,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,48.844975,2.358313,2019.0,0
4,Apis,Apis mellifera,L'Abeille mellifère,48.844975,2.358313,2019.0,0
8,Apis,Apis mellifera,L'Abeille mellifère,47.679478,-3.183325,2019.0,-1
...,...,...,...,...,...,...,...
670674,Bombus,Bombus,Les Bourdons à pilosité fauve à grise,44.711632,4.567840,2023.0,4
670677,Bombus,Bombus,Les Bourdons à pilosité fauve à grise,44.711498,4.567912,2023.0,4
670727,Bombus,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,48.218102,-1.693974,2023.0,1
670740,Bombus,Bombus,Les Bourdons à pilosité fauve à grise,44.712618,4.568999,2023.0,4


In [16]:
!pip install scikit-mobility


  Obtaining dependency information for scikit-mobility from https://files.pythonhosted.org/packages/bc/6e/5a5ac81e5408426754d217463eca563a8bc8e6b2c221ff4424e5b9282b77/scikit_mobility-1.3.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/95.0 kB ? eta -:--:--
     ---------------------------------------- 95.0/95.0 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     --------------------- ------------------ 0.5/1.0 MB 11.3 MB/s eta 0:00:01
     ---------------------------------------  1.0/1.0 MB 10.8 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 8.2 MB/s eta 0:00:00
  Obtaining dependency information for h3<4.0.0,>=3.7.3 from https://files.pythonhosted.org/packages/80/0c/22ec0172f390f79684f8463f34ba0b7f6f7ed1997325e0d2ff2a6632a877/h3-3.7.6-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pooch<2.0.0,>=1.6.0 from https://files.pythonhosted.org/packages/f4/72/8ae0

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [48 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\clustering.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\configuration.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\cut.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\datatypes.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\formula.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\layout.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\matching.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\operators.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\igraph\sparse_matrix.py -> build\lib.win-amd64-cpython-311\igraph
  copying src\

échec de l'installation : pas possible pour moi d'installer Cmake

In [19]:
import pandas as pd
from skmob import TrajDataFrame
from skmob import algorithms, utils

ModuleNotFoundError: No module named 'skmob'

In [ ]:
# Chargement des données de trajectoires spatio-temporelles
# Supposons que vous ayez un DataFrame appelé 'df' contenant des données de trajectoires
# avec les colonnes 'user_id', 'lat', 'lng' pour les coordonnées spatiales et 'timestamp' pour le temps
# Assurez-vous que 'timestamp' est au format datetime
tdf = TrajDataFrame(df, latitude='latitude', longitude='longitude', datetime='timestamp', user_id='user_id')

# Paramètres pour ST-DBSCAN
epsilon = 0.01  # Rayon de recherche spatial (en degrés)
min_samples = 10  # Nombre minimal de points dans un cluster

# Exécution de ST-DBSCAN
stdbscan_clusters = algorithms.st_dbscan(tdf, epsilon=epsilon, min_samples=min_samples)

# Affichage des résultats
print(stdbscan_clusters)